# Model training

In [1]:
import torch
from torch import nn
import pandas as pd

In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# Load data

X_train = pd.read_csv("./../data/features/in_features_train.csv")
X_train_pca = pd.read_csv("./../data/features/pca_in_features_train.csv")

X_train.shape, X_train_pca.shape

((145, 2593), (145, 61))

In [4]:
y_train = pd.read_csv("./../data/features/out_features_train.csv").label
y_train_pca = pd.read_csv("./../data/features/pca_out_features_train.csv").label

y_train.shape, y_train_pca.shape

((145,), (145,))

In [5]:
X_test = pd.read_csv("./../data/features/in_features_test.csv")
X_test_pca = pd.read_csv("./../data/features/pca_in_features_test.csv")

X_test.shape, X_test_pca.shape

((16, 2593), (16, 61))

In [6]:
y_test = pd.read_csv("./../data/features/out_features_test.csv").label

y_test.shape

(16,)

## Support Vector Machine (SVM)

### Non PCA features

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold

svm = SVC(kernel='rbf', C=10, gamma=0.01)
# p_grid = {"C": [1, 10, 20, 50, 100], "gamma": [0.01, 0.1, 0.2, 0.4, 0.6, 0.8]}
# cv = KFold(n_splits=5, shuffle=True)

# clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=cv, scoring="f1_macro")
# clf.fit(X_train, y_train)

clf = svm.fit(X_train, y_train)

In [8]:
y_pred = clf.predict(X_test)
y_pred

array([0, 0, 0, 0, 2, 0, 0, 2, 1, 2, 0, 0, 1, 0, 0, 0], dtype=int64)

In [9]:
y_test.to_numpy()

array([2, 2, 0, 0, 0, 2, 0, 1, 3, 1, 2, 0, 1, 0, 0, 0], dtype=int64)

In [10]:
from modules.engine import get_metrics
svm_train_metrics = get_metrics(y_train, clf.predict(X_train))

Accuracy:  0.8689655172413793  | F1-score:  0.8692980296196385  | Precision:  0.8928321678321678  | Recall:  0.8689655172413794


In [11]:
# Evaluate
svm_test_metrics = get_metrics(y_test, y_pred)

Accuracy:  0.5  | F1-score:  0.28421052631578947  | Precision:  0.28409090909090906  | Recall:  0.3020833333333333


In [12]:
clf.get_params()

{'C': 10,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.01,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [13]:
import joblib
from pathlib import Path


# Create models directory (if it doesn't already exist), see: https://docs.python.org/3/library/pathlib.html#pathlib.Path.mkdir
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # create parent directories if needed
                 exist_ok=True # if models directory already exists, don't error
)

# Create model save path
MODEL_NAME = "svm.pk"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

joblib.dump(clf, MODEL_SAVE_PATH)

['models\\svm.pk']

### PCA features


In [14]:
svm = SVC(kernel='rbf', C=20, gamma=0.01)
# p_grid = {"C": [1, 10, 100], "gamma": [0.01, 0.1]}
# cv = KFold(n_splits=5, shuffle=True)

# clf_pca = GridSearchCV(estimator=svm, param_grid=p_grid, cv=cv, scoring="f1_macro")
# clf_pca.fit(X_train_pca, y_train)

clf_pca = svm.fit(X_train_pca, y_train_pca)

In [15]:
y_pred = clf_pca.predict(X_test_pca)

In [16]:
y_test.to_numpy()

array([2, 2, 0, 0, 0, 2, 0, 1, 3, 1, 2, 0, 1, 0, 0, 0], dtype=int64)

In [17]:
svm_pca_train_metrics = get_metrics(y_train, clf_pca.predict(X_train_pca))

Accuracy:  0.9793103448275862  | F1-score:  0.9791873019779139  | Precision:  0.9792939244663383  | Recall:  0.9793103448275862


In [18]:
# Evaluate
svm_pca_test_metrics = get_metrics(y_test, y_pred)

Accuracy:  0.5  | F1-score:  0.27777777777777773  | Precision:  0.2583333333333333  | Recall:  0.3020833333333333


## Linear model

## Non-PCA

In [19]:
X_train = torch.from_numpy(X_train.to_numpy()).type(torch.float)
X_test = torch.from_numpy(X_test.to_numpy()).type(torch.float)

classes = y_train.unique()

y_train = torch.from_numpy(y_train.to_numpy()).type(torch.LongTensor)
y_test = torch.from_numpy(y_test.to_numpy()).type(torch.LongTensor)

in_features, out_features = X_train.shape[1], len(classes)
in_features, out_features

(2593, 5)

In [20]:
from modules.model_builder import BaseModel

model_0 = BaseModel(in_features, 50, out_features).to(device)

In [21]:
from modules.engine import train, save_model

loss_fn = nn.CrossEntropyLoss()
optimizer_class = torch.optim.SGD

train(model_0, X_train, X_test, y_train, y_test,
      loss_fn, optimizer_class, epochs=500, lr=0.1, device=device, verbose=True)

Epoch: 0
Accuracy:  0.0  | F1-score:  0.0  | Precision:  0.0  | Recall:  0.0
Train Loss: 1.6107858419418335 | Test loss: 1.6422420740127563
---------------------------------------------------------
Epoch: 100
Accuracy:  0.3125  | F1-score:  0.17230769230769233  | Precision:  0.20833333333333331  | Recall:  0.17714285714285713
Train Loss: 1.5866611003875732 | Test loss: 1.6073089838027954
---------------------------------------------------------
Epoch: 200
Accuracy:  0.1875  | F1-score:  0.2  | Precision:  0.14583333333333331  | Recall:  0.375
Train Loss: 1.318030595779419 | Test loss: 1.542609691619873
---------------------------------------------------------
Epoch: 300
Accuracy:  0.3125  | F1-score:  0.2602564102564103  | Precision:  0.2708333333333333  | Recall:  0.38888888888888884
Train Loss: 0.9446457624435425 | Test loss: 1.3985153436660767
---------------------------------------------------------
Epoch: 400
Accuracy:  0.4375  | F1-score:  0.3205128205128205  | Precision:  0.3333

In [22]:
model_0.eval()
with torch.inference_mode():
    y_train_logits = model_0(X_train.to(device))
    y_train_pred = y_train_logits.argmax(dim=1).to("cpu")
    y_test_logits = model_0(X_test.to(device))
    y_test_pred = y_test_logits.argmax(dim=1).to("cpu")

In [23]:

mpl_train_metrics = get_metrics(y_train, y_train_pred)

Accuracy:  0.9310344827586207  | F1-score:  0.9290653432588917  | Precision:  0.9333418334820859  | Recall:  0.9310344827586207


In [24]:

mpl_test_metrics = get_metrics(y_test, y_test_pred)

Accuracy:  0.4375  | F1-score:  0.30624999999999997  | Precision:  0.3229166666666667  | Recall:  0.3020833333333333


In [25]:
save_model(model_0, "mlp.pth")

Saving model to: models\mlp.pth


## PCA Model

In [26]:
X_train_pca = torch.from_numpy(X_train_pca.to_numpy()).type(torch.float)
X_test_pca = torch.from_numpy(X_test_pca.to_numpy()).type(torch.float)

y_train_pca = torch.from_numpy(y_train_pca.to_numpy()).type(torch.LongTensor)



In [27]:
in_features_pca = X_train_pca.shape[1]
in_features_pca, out_features

(61, 5)

In [28]:
model_0_pca = BaseModel(in_features_pca, 50, out_features).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer_class = torch.optim.SGD

train(model_0_pca, X_train_pca, X_test_pca, y_train, y_test,
      loss_fn, optimizer_class, 500, 0.1, device, verbose=True)

Epoch: 0
Accuracy:  0.0625  | F1-score:  0.041666666666666664  | Precision:  0.03125  | Recall:  0.0625
Train Loss: 1.6101586818695068 | Test loss: 1.592514991760254
---------------------------------------------------------
Epoch: 100
Accuracy:  0.1875  | F1-score:  0.08823529411764706  | Precision:  0.25  | Recall:  0.05357142857142857
Train Loss: 1.5574438571929932 | Test loss: 1.6029856204986572
---------------------------------------------------------
Epoch: 200
Accuracy:  0.1875  | F1-score:  0.09375  | Precision:  0.25  | Recall:  0.057692307692307696
Train Loss: 1.1535381078720093 | Test loss: 1.456896424293518
---------------------------------------------------------
Epoch: 300
Accuracy:  0.375  | F1-score:  0.2897727272727273  | Precision:  0.3020833333333333  | Recall:  0.37142857142857144
Train Loss: 0.8715500235557556 | Test loss: 1.3796861171722412
---------------------------------------------------------
Epoch: 400
Accuracy:  0.4375  | F1-score:  0.3242424242424242  | Pre

In [29]:
model_0_pca.eval()
with torch.inference_mode():
    y_train_logits = model_0_pca(X_train_pca.to(device))
    y_train_pred = y_train_logits.argmax(dim=1).to("cpu")
    y_test_logits = model_0_pca(X_test_pca.to(device))
    y_test_pred = y_test_logits.argmax(dim=1).to("cpu")

In [30]:
mpl_pca_train_metrics = get_metrics(y_train, y_train_pred)

Accuracy:  0.9793103448275862  | F1-score:  0.9793041926851027  | Precision:  0.9795238095238096  | Recall:  0.9793103448275862


In [31]:
mpl_pca_test_metrics = get_metrics(y_test, y_test_pred)

Accuracy:  0.5625  | F1-score:  0.38758169934640524  | Precision:  0.3916666666666666  | Recall:  0.3958333333333333


In [32]:
save_model(model_0_pca, "mlp_pca.pth")

Saving model to: models\mlp_pca.pth


## Convolution Model

### Non-PCA

In [33]:
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

In [34]:
in_channels, in_length = X_train.shape[1:]
in_channels, in_length

(1, 2593)

In [35]:
from modules.model_builder import ConvModel

model_1 = ConvModel(in_channels, in_length, 50, out_features).to(device)

In [36]:
train(model_1, X_train, X_test, y_train, y_test,
      loss_fn, optimizer_class, 100, 0.1, device, verbose=True)

Epoch: 0
Accuracy:  0.125  | F1-score:  0.14166666666666666  | Precision:  0.11458333333333333  | Recall:  0.1875
Train Loss: 1.6094942092895508 | Test loss: 1.6093770265579224
---------------------------------------------------------


In [37]:
model_1.eval()
with torch.inference_mode():
    y_train_logits = model_1(X_train.to(device))
    y_train_pred = y_train_logits.argmax(dim=1).to("cpu")
    y_test_logits = model_1(X_test.to(device))
    y_test_pred = y_test_logits.argmax(dim=1).to("cpu")

In [38]:
cnn_train_metrics = get_metrics(y_train, y_train_pred)

Accuracy:  0.9103448275862069  | F1-score:  0.908981485972997  | Precision:  0.9202953908836262  | Recall:  0.9103448275862067


In [39]:
cnn_test_metrics = get_metrics(y_test, y_test_pred)

Accuracy:  0.25  | F1-score:  0.17857142857142858  | Precision:  0.1875  | Recall:  0.17708333333333331


In [40]:
save_model(model_1, "cnn.pth")

Saving model to: models\cnn.pth


### PCA

In [41]:
X_train_pca = X_train_pca.unsqueeze(1)
X_test_pca = X_test_pca.unsqueeze(1)

In [42]:
in_channels, in_length = X_train_pca.shape[1:]
in_channels, in_length

(1, 61)

In [43]:
model_1_pca = ConvModel(in_channels, in_length, 70, out_features).to(device)

In [44]:
train(model_1_pca, X_train_pca, X_test_pca, y_train_pca, y_test,
      loss_fn, optimizer_class, 200, 0.1, device, verbose=True)

Epoch: 0
Accuracy:  0.25  | F1-score:  0.1  | Precision:  0.25  | Recall:  0.0625
Train Loss: 1.6102142333984375 | Test loss: 1.5997506380081177
---------------------------------------------------------
Epoch: 100
Accuracy:  0.5  | F1-score:  0.2666666666666667  | Precision:  0.3020833333333333  | Recall:  0.2596153846153846
Train Loss: 1.5971649885177612 | Test loss: 1.6066993474960327
---------------------------------------------------------


In [45]:
model_1_pca.eval()
with torch.inference_mode():
    y_train_logits = model_1_pca(X_train_pca.to(device))
    y_train_pred = y_train_logits.argmax(dim=1).to("cpu")
    y_test_logits = model_1_pca(X_test_pca.to(device))
    y_test_pred = y_test_logits.argmax(dim=1).to("cpu")

In [46]:
cnn_pca_train_metrics = get_metrics(y_train_pca, y_train_pred)

Accuracy:  0.7241379310344828  | F1-score:  0.7202254841899678  | Precision:  0.7579857397504457  | Recall:  0.7241379310344828


In [47]:
cnn_pca_test_metrics = get_metrics(y_test, y_test_pred)

Accuracy:  0.5  | F1-score:  0.25833333333333336  | Precision:  0.22916666666666669  | Recall:  0.3020833333333333


In [48]:
save_model(model_1, "cnn_pca.pth")

Saving model to: models\cnn_pca.pth
